In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [21]:

df = pd.read_parquet("fhv_tripdata_2021-01.parquet")
#df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
#df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

#df = df[(df.duration >= 1) & (df.duration <= 60)]

#categorical = ['PULocationID', 'DOLocationID']
#numerical = ['trip_distance']

#df[categorical] = df[categorical].astype(str)

In [22]:
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037
...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285


In [23]:
njan = df.shape
print(f"Number of records in January: {njan[0]}")

Number of records in January: 1154112


In [24]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)


In [25]:
mean_duration = df["duration"].mean()
print(f"Mean duration of trips in January: {mean_duration}")

Mean duration of trips in January: 19.167224093791006


In [26]:
# Dropping some records, keeping only those with 1 to 60 min duration
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [27]:
df.shape

(1109826, 8)

In [28]:
#Fraction of missing values to total number for pickup location
100*df['PUlocationID'].isna().sum()/df.shape[0]

83.52732770722618

In [29]:
## Replacing missing values
df['PUlocationID'] = df['PUlocationID'].fillna(-1)
df['DOlocationID'] = df['DOlocationID'].fillna(-1)

/tmp/ipykernel_6926/2624768413.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PUlocationID'] = df['PUlocationID'].fillna(-1)
/tmp/ipykernel_6926/2624768413.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOlocationID'] = df['DOlocationID'].fillna(-1)


In [30]:
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,None,B00037,9.050000
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,19.433333


In [49]:
features = ['PUlocationID', 'DOlocationID']
numerical = ['duration']


In [50]:
#train_dicts = df[categorical + numerical].to_dict(orient='records')
train_dicts = df[features].to_dict(orient='records')


dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)


y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

11.415432830521663

In [51]:
X_train.shape

(1109826, 2)

In [52]:
import numpy as np
score = np.sqrt(mean_squared_error(y_train, y_pred))
score

11.415432830521663

In [65]:
df_val = pd.read_parquet("fhv_tripdata_2021-02.parquet")

In [66]:
#do the same things for the previous df

In [74]:
df_val['duration'] = df_val.dropOff_datetime - df_val.pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]
df_val['PUlocationID'] = df_val['PUlocationID'].fillna(-1)
df_val['DOlocationID'] = df_val['DOlocationID'].fillna(-1)

In [75]:
df_val.shape

(990113, 8)

In [76]:
val_dicts = df_val[features].to_dict(orient='records')

X_val = dv.fit_transform(val_dicts)
y_val = df_test[target].values

In [77]:
y_pred = lr.predict(X_val)

In [80]:
#mean_squared_error(y_test, y_pred, squared=False)

In [81]:
mean_squared_error(y_val, y_pred, squared=False)

11.85822362355935

In [ ]:
#he old stuff

In [13]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [14]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.479513631630414

In [16]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [17]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.167275941179728